In [ ]:
from pprint import PrettyPrinter
import numpy as np
import pandas as pd
from pymongo import MongoClient
import json

In [ ]:
# Connect to Mongodb
client = MongoClient('mongodb://localhost:27017')

In [ ]:
db = client['riotGamesOlddb']
match_collection = db['matchdatas']

In [ ]:
to_get = {'matchSummaryData.info.participants':1,"_id":0,"title":1}
match_cursor = match_collection.find({"$and":[{'matchSummaryData.info.gameMode':"CLASSIC"},{'matchSummaryData.metadata.participants':{'$size':10}}]},to_get)


In [ ]:
participant_summary = []

for match in match_cursor:
    participant_summary.append(match['matchSummaryData']['info']['participants'])
print(len(participant_summary))
participant_summary[0]

Dataframe creation:

In [ ]:
p_df = pd.json_normalize(json.loads((pd.DataFrame(participant_summary)).to_json(orient="records")))

p_df.head(2)

In [ ]:
name = []

for column in p_df.filter(regex='(summonerName)$', axis="columns"):
    this_column_values = p_df.filter(regex='(summonerName)$', axis="columns")[column].tolist()
    name += this_column_values

champ_id = []

for column in p_df.filter(regex='(championId)$', axis="columns"):
    this_column_values = p_df.filter(regex='(championId)$', axis="columns")[column].tolist()
    champ_id += this_column_values

In [ ]:
for i in range(len(name)):
    if name[i] =='':
        print(str(i) + ' is empty')

for i in range(len(champ_id)):
    if champ_id[i] =='':
        print(str(i) + ' is empty')

In [ ]:
for x in range(len(name)):
    print(name[x],champ_id[x])

In [ ]:
client = MongoClient('mongodb://localhost:27017')
db = client['riotGamesOlddb']
account_collection = db['accountdatas']

championMasteryPoints = []
for x in range(len(name)):
    docs = account_collection.aggregate([{"$match": {"title": name[x], "champData": { "$elemMatch": { "$and": [{ "championId": champ_id[x] }] } } } }, { "$project": {"title":1, "champData": { "$filter": { "input": "$champData", "as": "champData", "cond": { "$and": [{ "$eq": ["$$champData.championId", champ_id[x]] }] } } } } }])
    for doc in docs:
        championMasteryPoints.append([doc['champData'][0]['championPoints']])

Data Processing - Remove unnecessary columns:

In [ ]:
to_remove = ['championName', 'championTransform', 'eligibleForProgression', 'gameEndedInEarlySurrender', 'gameEndedInSurrender', 'individualPosition', 'lane', 'nexusKills', 'nexusLost', 'nexusTakedowns', 'participantId', 'profileIcon', 'puuid', 'riotIdName', 'riotIdTagline', 'role', 'summonerId', 'teamEarlySurrendered', 'teamId', 'teamPosition'
, 'unrealKills', 'challenges.blastConeOppositeOpponentCount', 'challenges.completeSupportQuestInTime', 'challenges.controlWardTimeCoverageInRiverOrEnemyHalf', 'challenges.dancedWithRiftHerald', 'challenges.dodgeSkillShotsSmallWindow', 'challenges.doubleAces', 'challenges.elderDragonKillsWithOpposingSoul', 'challenges.epicMonsterKillsNearEnemyJungler', 'challenges.epicMonsterKillsWithin30SecondsOfSpawn', 'challenges.epicMonsterStolenWithoutSmite', 'challenges.firstTurretKilledTime', 'challenges.flawlessAces', 'challenges.hadOpenNexus', 'challenges.immobilizeAndKillWithAlly', 'challenges.initialBuffCount', 'challenges.initialCrabCount', 'challenges.junglerTakedownsNearDamagedEpicMonster', 'challenges.killAfterHiddenWithAlly', 'challenges.killedChampTookFullTeamDamageSurvived', 'challenges.killsNearEnemyTurret', 'challenges.killsOnRecentlyHealedByAramPack', 'challenges.killsUnderOwnTurret', 'challenges.killsWithHelpFromEpicMonster', 'challenges.knockEnemyIntoTeamAndKill', 'challenges.lostAnInhibitor', 'challenges.moreEnemyJungleThanOpponent', 'challenges.multiKillOneSpell', 'challenges.multiTurretRiftHeraldCount', 'challenges.multikillsAfterAggressiveFlash', 'challenges.outerTurretExecutesBefore10Minutes', 'challenges.outnumberedKills', 'challenges.outnumberedNexusKill', 'challenges.perfectDragonSoulsTaken', 'challenges.perfectGame', 'challenges.pickKillWithAlly', 'challenges.playedChampSelectPosition', 'challenges.poroExplosions', 'challenges.quickCleanse', 'challenges.quickFirstTurret', 'challenges.quickSoloKills', 'challenges.saveAllyFromDeath', 'challenges.scuttleCrabKills', 'challenges.snowballsHit', 'challenges.soloBaronKills', 'challenges.survivedSingleDigitHpCount', 'challenges.survivedThreeImmobilizesInFight', 'challenges.takedownOnFirstTurret', 'challenges.takedownsAfterGainingLevelAdvantage', 'challenges.takedownsFirstXMinutes', 'challenges.takedownsInAlcove', 'challenges.takedownsInEnemyFountain', 'challenges.threeWardsOneSweeperCount', 'challenges.tookLargeDamageSurvived', 'challenges.turretsTakenWithRiftHerald', 'challenges.twentyMinionsIn3SecondsCount', 'challenges.unseenRecalls', 'challenges.wardsGuarded', 'perks.statPerks.defense', 'perks.statPerks.flex', 'perks.statPerks.offense', 'perks.styles', 'challenges.earliestDragonTakedown', 'challenges.highestCrowdControlScore', 'challenges.junglerKillsEarlyJungle', 'challenges.highestChampionDamage', 'challenges.fasterSupportQuestCompletion', 'challenges.earliestBaron', 'challenges.highestWardKills']


In [ ]:
remove_cols = []

for e in to_remove:
    for i in range(10):
        remove_cols.append(str(i)+'.'+e)
len(remove_cols)

In [ ]:
p_df = p_df.drop(remove_cols,axis=1)

Data Processing - Remove individual player win and add team win

In [ ]:
p_df['winningTeam'] = np.where(p_df['0.win'] == 1, 1, 2)

In [ ]:
remove_wins = []

for i in range(10):
    remove_wins.append(str(i)+'.win')
p_df = p_df.drop(remove_wins,axis=1)

Data Processing - Replace NA with 0:

In [ ]:
p_df = p_df.replace(np.nan, 0)

Data Processing - Replace bool with int:


In [ ]:
bool_cols = p_df.select_dtypes(include=['bool']).columns

for col in bool_cols:
    p_df[col]=p_df[col].astype(int)


In [ ]:
p_df.head(5)

Data Processing: Create Summoner codes

In [ ]:
for i in range(10):
    p_df[ str(i) +'.summonerCode'] = p_df[ str(i) +'.summonerName'].astype("category").cat.codes


In [ ]:
corr = p_df.corr()

In [ ]:
corr['winningTeam']